Current example follows this [document](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html) .

In [1]:
# 1. Prepare dependencies
import mnist

import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn


In [2]:
# 2. prepare training and testing dataloader

mnist.datasets_url="http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/"

train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()


train_images_tensor = torch.Tensor(train_images)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.int64)
test_images_tensor = torch.Tensor(test_images)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.int64)

train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)


train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU(),
            nn.Linear(10, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
    (3): ReLU()
    (4): Linear(in_features=10, out_features=10, bias=True)
  )
)

In [4]:
# Define hyperparameters

learning_rate = 1e-3
batch_size = 64
epochs = 5

In [5]:
# Define the train_loop function

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        
        
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')


def test_loop(dataloader, model, loss_fn):
    
    model.eval()

    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y ).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /=size

    print(f'Test Error: \n Accurary: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')






In [6]:
# 3. training model

loss_fn = nn.CrossEntropyLoss()

optimizer =torch.optim.SGD(model.parameters(), lr=learning_rate)


epochs = 10

for t in range(epochs):
    print(f'Epoch {t+1}\n--------------------')
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)




Epoch 1
--------------------
loss: 6.901670 [   64/60000]
loss: 2.143878 [ 6464/60000]
loss: 1.919856 [12864/60000]
loss: 2.145686 [19264/60000]
loss: 1.796265 [25664/60000]
loss: 1.943457 [32064/60000]
loss: 1.869414 [38464/60000]
loss: 1.683442 [44864/60000]
loss: 1.788408 [51264/60000]
loss: 1.704232 [57664/60000]
Test Error: 
 Accurary: 29.4%, Avg loss: 1.745113 

Epoch 2
--------------------
loss: 1.722206 [   64/60000]
loss: 1.736794 [ 6464/60000]
loss: 1.473343 [12864/60000]
loss: 1.474146 [19264/60000]
loss: 1.558322 [25664/60000]
loss: 1.603696 [32064/60000]
loss: 1.539950 [38464/60000]
loss: 1.529979 [44864/60000]
loss: 1.267327 [51264/60000]
loss: 1.457645 [57664/60000]
Test Error: 
 Accurary: 46.1%, Avg loss: 1.416557 

Epoch 3
--------------------
loss: 1.396255 [   64/60000]
loss: 1.270390 [ 6464/60000]
loss: 0.970714 [12864/60000]
loss: 1.023707 [19264/60000]
loss: 1.398977 [25664/60000]
loss: 1.093814 [32064/60000]
loss: 1.005096 [38464/60000]
loss: 1.192238 [44864/6000

In [8]:
# 4. save model
# with both
# - model architecture
# - and, state_dict

torch.save(model, './model/main.pth')
